<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr1/blob/Kozlov-Alexey-folder/Kozlov%20Alexey/InspectrumClinic_gr1_akozlov_20230515_02week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Рекомендательная система для профпатолога (INSPECTRUM CLINIC)**

Цель: 
Разработать прототип системы, реализующей функцию рекомендательной системы для врача профпатолога.

Алексей Козлов 2023.05.15 - 2023.05.22 (2-я неделя):
обработка данных, создание модели DL-точности первой точности

In [2]:
# Загрузка файлов из облака
import gdown
# Библиотека для работы с массивами данных
import numpy as np
# Библиотека для обработки и анализа данных
import pandas as pd
# Библиотека для построения графиков
import matplotlib.pyplot as plt
# Команда для отображения графики в ячейках среды Google Colaboratory
%matplotlib inline
# Библиотека для построения графиков
import seaborn as sns

# Подключение класса создания модели Sequential
# Sequential – модуль для создания последовательной модели нейронной сети
from tensorflow.keras.models import Sequential
# Подключение класса Dense - полносвязный слой
# Dense – линейный (полносвязный) слой. Из таких слоев будет создана ваша нейросеть
from tensorflow.keras.layers import Dense
# utils – модуль с полезными инструментами для подготовки данных
from tensorflow.keras import utils

In [3]:
# Загрузка файла датасета
url1 = 'https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx'
url2 = '/content/InspectrumClinic1.xlsx'
gdown.download(url1, url2, quiet=False)

Downloading...
From: https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx
To: /content/InspectrumClinic1.xlsx
100%|██████████| 32.4M/32.4M [00:00<00:00, 38.4MB/s]


'/content/InspectrumClinic1.xlsx'

In [4]:
# Чтение данных (df - DataFrame)
df = pd.read_excel(url2)

---
**Шаг 1. Обработка и анализ данных**
---

In [5]:
# Выведем информацию о заполненности ячеек каждого признака
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982 entries, 0 to 1981
Columns: 4587 entries, ДокументПрохождениеМедосмотра to Офтальмология1_ДвигательныйАппаратГлазДополнение_ЗначениеПредставление
dtypes: bool(1), datetime64[ns](43), float64(1253), object(3290)
memory usage: 69.3+ MB


In [6]:
# Блок переменных

# Задание константы количества распознаваемых классов
CLASS_COUNT = 3
AGE_CLASS_COUNT = 11
GENDER_CLASS_COUNT = 2

# Процентное соотношение тренировочной и тестовых выборок
# например: 80% на обучающую выборку и 20% на тестовую
trainPercent = 80

# Названия колонок, которые используются для обучения(для работы нейросети)
targetColName = 'ЗаключениеМК'
dt1ColName = 'КлиентДатаРождения'
dt2ColName = 'ДатаЗавершенияМедосмотра'
genderColName = 'КлиентПол'
psychColName = 'ПсихОсвидетельствование' # Psych Examination
harmfulColName = 'ВредныеФакторы'
psychnark1_MKB101 = 'ПсихиатрияНаркология1_МКБ101'

xTrainList = list()
xTestList = list()

#~~~~~~~~~~~~~~~~~~~~~~~~
print(f'CLASS_COUNT: {CLASS_COUNT}')
print(f'AGE_CLASS_COUNT: {AGE_CLASS_COUNT}')
print(f'GENDER_CLASS_COUNT: {GENDER_CLASS_COUNT}')

print('~'*50)
print(f'trainPercent: {trainPercent}')
print('~'*50)
print(f'targetColName: |{targetColName}|')
print(f'dt1ColName: |{dt1ColName}|')
print(f'dt2ColName: |{dt2ColName}|')
print(f'genderColName: |{genderColName}|')
print(f'psychColName: |{psychColName}|')
print(f'harmfulColName: |{harmfulColName}|')
print(f'psychnark1_MKB101: |{psychnark1_MKB101}|')
print('~'*50)
print(f'xTrainList: {xTrainList}')
print(f'xTestList: {xTestList}')

CLASS_COUNT: 3
AGE_CLASS_COUNT: 11
GENDER_CLASS_COUNT: 2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
trainPercent: 80
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
targetColName: |ЗаключениеМК|
dt1ColName: |КлиентДатаРождения|
dt2ColName: |ДатаЗавершенияМедосмотра|
genderColName: |КлиентПол|
psychColName: |ПсихОсвидетельствование|
harmfulColName: |ВредныеФакторы|
psychnark1_MKB101: |ПсихиатрияНаркология1_МКБ101|
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
xTrainList: []
xTestList: []


**Функции обработки данных:**

In [7]:
def printRowColCount():
  '''
    Отображаем текущее количества строк и столбцов в датасете
  '''
  print(f'Текущий датасет: число строк: {df.shape[0]}, число колонок: {df.shape[1]}, {df.shape}')

def getColInx(colName):
  '''
    Получение индекса колонки
    вход:
        colName - название колонки
    выход:
        retVal - индекс колонки
  '''
  retVal = df.columns.get_loc(colName)
  return retVal

def getColVals(colInx):
  '''
    Получение списка значений колонки
    вход:
        colInx - индекс колонки
    выход:
        retVal - список значений
  '''
  retVal = df.iloc[:, colInx].tolist()
  return retVal

def getColVals_ByName(colName):
  '''
    Получение списка значений колонки
    вход:
        colName - название колонки
    выход:
        retVal - список значений
  '''
  inx = getColInx(colName)
  retVal = getColVals(inx)
  return retVal

def getColUniqueVals(colName):
  '''
    Получение списка уникальных значений колонки
    вход:
        colName - название колонки
    выход:
        retVal - список уникальных значений
  '''
  retVal = df[colName].unique().tolist()
  return retVal

def getBoolColNames():
  '''
    Получение списка названий колонок типа bool
  '''
  retVal = list(df.select_dtypes(include=['bool']).columns)
  return retVal

def getDateTimeColNames():
  '''
    Получение списка названий колонок типа datetime64
  '''
  # retVal = list(df.select_dtypes(include=['datetime64[ns]']).columns)
  retVal = list(df.select_dtypes(include=['datetime64']).columns)
  return retVal

def getObjStrColNames():
  '''
    Получение списка названий колонок типа object-string
  '''
  retVal = list(df.select_dtypes(include=['object']).columns)
  return retVal

# def patchTextCol(colName):
#   '''
#     Удаляем в описаниях(текстовых полях) неразрывные пробелы, перевод каретки: \xa0 \n
#     вход:
#         colName - название колонки
#   '''
#   # Цикл по всем строкам
#   for i in range(df.shape[0]):
#     #заменяем фрагменты кодировки ASCII на отсутствие символов
#     df.values[i][j] = df.values[i][j].replace("\xa0","")
#     df.values[i][j] = df.values[i][j].replace("\n"," ")
# #TODO
# #    привести к нижнему регистру и удалить пробелы слева и справа

def print1Row(rowInx):
  '''
    Печать одной строки с индексом rowInx
    вход:
        rowInx - индекс строки
  '''
  # Цикл по всем колонкам
  for j in range(df.shape[1]):
    print(f'{j}: {df.values[rowInx][j]}')

def print1RowCol14(rowInx):
  '''
    Печать 14-и колонок одной строки с индексом rowInx
    вход:
        rowInx - индекс строки
  '''
  # Цикл по всем колонкам
  for j in range(14):
    print(f'{j}: {df.values[rowInx][j]}')

def printColNames(colNames):
  '''
    Печать названий колонок по списку
    вход:
        colNames - список названий колонок
  '''
  ilen = len(colNames)
  for icol in range(ilen):
    print(f'col:{icol} |{colNames[icol]}|')

def getForbiddenRowInxs(colName, forbiddenVals):
  '''
    Получение индексов запрещенных строк в датасете
    вход:
        colName - название колонки
        forbiddenVals - список запрещенных значений
    выход:
        retVal - список индексов строк
  '''
  retVal = df[df[colName].isin(forbiddenVals)].index
  return retVal

def getNanRowInxs(colName):
  '''
    Получение индексов пустых строк в датасете
    вход:
        colName - название колонки
    выход:
        retVal - список индексов строк
  '''
  retVal = df[df[colName].isnull()].index
  return retVal

def removeCols(colNames, df):
  '''
    Удаление в датасете колонок
    вход:
        colNames - список названий колонок
  '''
  for rcol in colNames:
    if rcol in df.columns:
      df.drop(rcol, axis=1, inplace=True)

  return df

def removeRows(rowInxs, df):
  '''
    Удаление в датасете строк
    вход:
        rowInxs - индексы строк для удаления
  '''
  if (len(rowInxs) > 0):
    df.drop(rowInxs, axis=0, inplace=True)
  return df

def getColumnsWithString(str):
  '''
    Возвращает список названий колонок, в которых содержится заданная строка
    вход:
        str - часть названия колонки, например мкб
    выход:
        retVal - список названий колонок, содержащих заданную строку
  '''
  totalCols = list(df.columns)
  retVal = [col for col in totalCols if str in col]
  return retVal



**Функции представления данных:**

In [8]:
# ЗаключениеМК
def getTargetCategory(arg):
  '''
    Получение целевого категориального значения по текстовому описанию
    вход (arg):
        ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден']
    выход:
        retVal: 0 - не годен, 1 - годен с ограничениями, 2 - годен
  '''
  if ('ГоденСКоррекциейЗрения' == arg) or ('ГоденБезРаботНаВысотах' == arg) or ('ОграниченноГоден' == arg):
    retVal = 1
  elif ('НуждаетсяВДообследованииИЛечении' == arg) or ('ВременноНегоден' == arg):
    retVal = 0
  else:
    retVal = 2

  return retVal


def getYTrainTestCategorical(colName, trnInxs, tstInxs):
  '''
    Создание целевого категориального массива для обучения сетки
    вход:
        colName - название целевой колонки
        trnInxs - индексы для обучения
        tstInxs - индексы для проверки и для обратного переобучения
    выход:
        два категориальных массива: train и test
  '''
  colVals = getColVals_ByName(colName)

  vecSize1 = trnInxs.size
  vecSize2 = tstInxs.size

  y_train = np.zeros(vecSize1, dtype=int)
  y_test = np.zeros(vecSize2, dtype=int)

  # Заполняем обучающий массив
  for i in range(vecSize1):
    inx = trnInxs[i]
    y_train[i] = getTargetCategory(colVals[inx])

  # Заполняем тестовый массив
  for i in range(vecSize2):
    inx = tstInxs[i]
    y_test[i] = getTargetCategory(colVals[inx])

  # Преобразование ответов в формат one_hot_encoding
  y_train2 = utils.to_categorical(y_train, CLASS_COUNT)
  y_test2 = utils.to_categorical(y_test, CLASS_COUNT)
  return (y_train2, y_test2)

In [9]:
# Возраст пациента
def getAgeCategory(arg):
  '''
    Возраст превращаем в категориальный: до 18лет, далее через 5, всего 11 классов
    вход (arg):
        arg - возраст в годах
    выход:
        retVal: индекс категории: 0-17: 0, 18-22: 1, 23-27: 2, ...., 73..: 10
  '''
  age2 = int((arg-13)/5)
  retVal = max(0, min((AGE_CLASS_COUNT-1),age2))
  return retVal


def getAgeTrainTestCategorical(birthDateColName, checkDateColName, trnInxs, tstInxs):
  '''
    Расчет возраста при проведении медосмотра
    Возраст превращаем в категориальный: до 18лет, далее через 5, всего 11 классов
    вход:
        birthDateColName - название колонки даты рождения
        checkDateColName - название колонки даты проведении медосмотра
        trnInxs - индексы для обучения
        tstInxs - индексы для проверки и для обратного переобучения
    выход:
        два категориальных массива: train и test
  '''

  dt1 = pd.to_datetime(df[birthDateColName])
  dt2 = pd.to_datetime(df[checkDateColName])
  dt12 = (dt2 - dt1).dt.days // 365
  #print(dt12)

  vecSize1 = trnInxs.size
  vecSize2 = tstInxs.size
  x_train = np.zeros(vecSize1, dtype=int)
  x_test = np.zeros(vecSize2, dtype=int)
  #print(x_train)
  #print(x_test)

  # Заполняем обучающий массив
  for i in range(vecSize1):
    inx = trnInxs[i]
    x_train[i] = getAgeCategory(dt12[inx])

  # Заполняем тестовый массив
  for i in range(vecSize2):
    inx = tstInxs[i]
    x_test[i] = getAgeCategory(dt12[inx])

  #print(x_train)
  #print(x_test)
  # Преобразование ответов в формат one_hot_encoding
  x_train2 = utils.to_categorical(x_train, AGE_CLASS_COUNT)
  x_test2 = utils.to_categorical(x_test, AGE_CLASS_COUNT)
  return (x_train2, x_test2)

In [10]:
# Клиент Пол
def getGenderCategory(arg):
  '''
    Категориальный пол клиента
    вход (arg):
        arg - уникальные значения: ['Женский', 'Мужской']
    выход:
        retVal: 0 - Женский, 1 - Мужской
  '''
  if 'Женский' == arg:
    retVal = 0
  else:
    retVal = 1
  return retVal

def getXTrainTestCategorical(colName, trnInxs, tstInxs):
  '''
    Создание целевого категориального массива для обучения сетки
    вход:
        colName - название целевой колонки
        trnInxs - индексы для обучения
        tstInxs - индексы для проверки и для обратного переобучения
        clsCount - число категориальных классов
    выход:
        два категориальных массива: train и test
  '''
  modeTrain = 0
  clsCount = 0
  if genderColName == colName:
    modeTrain = 1
    clsCount = GENDER_CLASS_COUNT

  colVals = getColVals_ByName(colName)

  vecSize1 = trnInxs.size
  vecSize2 = tstInxs.size

  y_train = np.zeros(vecSize1, dtype=int)
  y_test = np.zeros(vecSize2, dtype=int)

  # Заполняем обучающий массив
  for i in range(vecSize1):
    inx = trnInxs[i]
    if (1 == modeTrain):
      y_train[i] = getGenderCategory(colVals[inx])

  # Заполняем тестовый массив
  for i in range(vecSize2):
    inx = tstInxs[i]
    if (1 == modeTrain):
      y_test[i] = getGenderCategory(colVals[inx])

  # Преобразование ответов в формат one_hot_encoding
  y_train2 = utils.to_categorical(y_train, clsCount)
  y_test2 = utils.to_categorical(y_test, clsCount)
  return (y_train2, y_test2)

In [11]:
def addVecsToXTrainTest(x_train, x_test):
  '''
    Добавление данных по одному параметру
    вход:
        x_train,x_test - обучающие и тестовые выборки по одному парметру
  '''
  xTrainLen = len(xTrainList)
  xTestLen = len(xTestList)
  addFlag = False
  if 0 == xTrainLen:
    addFlag = True

  # print(f'xTrainList: {xTrainList}')
  # print(f'xTestList: {xTestList}')
  # print(f'xTrainLen: {xTrainLen}')
  # print(f'xTestLen: {xTestLen}')
  # print(f'addFlag: {addFlag}')

  #print(f'Количество осей: {x_train.ndim}')
  #print(f'Форма массива: {x_train.shape}')
  #print(f'Количество значений: {x_train.size}')
  #print(f'Количество осей: {x_test.ndim}')
  #print(f'Форма массива: {x_test.shape}')
  #print(f'Количество значений: {x_test.size}')

  trainRowCount, trainColCount = x_train.shape
  testRowCount, testColCount = x_test.shape
  # print(f'train число: строк: {trainRowCount}, столбцов: {trainColCount}')
  # print(f'test число: строк: {testRowCount}, столбцов: {testRowCount}')

  for i in range(trainRowCount):
    trainLine = list(x_train[i])
    if True == addFlag:
      xTrainList.append(trainLine)
    else:
      trainList = xTrainList[i]
      xTrainList[i] = trainList + trainLine

  for i in range(testRowCount):
    testLine = list(x_test[i])
    if True == addFlag:
      xTestList.append(testLine)
    else:
      testList = xTestList[i]
      xTestList[i] = testList + testLine


In [12]:
printRowColCount()
print1RowCol14(843)

Текущий датасет: число строк: 1982, число колонок: 4587, (1982, 4587)
0: 7b644813-5fdd-11ed-8109-0cc47aab8067
1: 956edb54-4047-11ec-80fa-0cc47aab8067
2: 1979-05-06 00:00:00
3: Мужской
4: Мужской
5: 2022-11-09 00:00:00
6: Периодический
7: Периодический
8: False
9: 00bbe942-2944-11ea-80d7-0cc47aab8067
10: nan
11: Слесарь-ремонтник
12: машинист бульдозера
13: тракторист


**Целевая колонка “ЗаключениеМК” типа object-string**

In [13]:
strCols = getObjStrColNames()
if targetColName in strCols:
    print(f'"{targetColName}": найдено в списке колонок типа object-string')
else:
    print(f'"{targetColName}": не найдено в списке колонок типа object-string')

сolUVals = getColUniqueVals(targetColName)
print(f'Список уникальных значений целевой колонки: {сolUVals}')

"ЗаключениеМК": найдено в списке колонок типа object-string
Список уникальных значений целевой колонки: ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден', nan, 'машинист бульдозера', 'Газоспасатель', ' Газоспасатель']


In [14]:
print('Размер датасета до удаления строк с заперещенными и пустыми значениями:')
printRowColCount()

# Удаление строк с запрещенными значениями
forbiddenVals = list()
forbiddenVals.append('машинист бульдозера')
forbiddenVals.append('Газоспасатель')
forbiddenVals.append(' Газоспасатель')
forbiddenRowInxs = getForbiddenRowInxs(targetColName, forbiddenVals)
df = removeRows(forbiddenRowInxs, df)

# Удаление строк c пустыми значениями
nanRowInxs = getNanRowInxs(targetColName)
df = removeRows(nanRowInxs, df)

print('Размер датасета после удаления строк:')
printRowColCount()
сolUVals = getColUniqueVals(targetColName)
print(f'Список уникальных значений целевой колонки: {сolUVals}')

Размер датасета до удаления строк с заперещенными и пустыми значениями:
Текущий датасет: число строк: 1982, число колонок: 4587, (1982, 4587)
Размер датасета после удаления строк:
Текущий датасет: число строк: 1571, число колонок: 4587, (1571, 4587)
Список уникальных значений целевой колонки: ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден']


**Колонки “КлиентДатаРождения”, “ДатаЗавершенияМедосмотра” типа datetime**

In [15]:
# Получаем список всех колонок типа datetime
datetimeCols = getDateTimeColNames()
printColNames(datetimeCols)

col:0 |КлиентДатаРождения|
col:1 |ДатаЗавершенияМедосмотра|
col:2 |ЛабораторныеИсследования1_ДатаПриема|
col:3 |Рентгенология1_ДатаПриема|
col:4 |Рентгенология2_ДатаПриема|
col:5 |ПсихиатрияНаркология1_ДатаПриема|
col:6 |Офтальмология1_ДатаПриема|
col:7 |Оториноларингология1_ДатаПриема|
col:8 |ХирургНеИсп1_ДатаПриема|
col:9 |Гинекология1_ДатаПриема|
col:10 |ФункциональнаяДиагностика1_ДатаПриема|
col:11 |Стоматология1_ДатаПриема|
col:12 |Эндоскопия1_ДатаПриема|
col:13 |Терапия1_ДатаПриема|
col:14 |Дерматовенерология1_ДатаПриема|
col:15 |Неврология1_ДатаПриема|
col:16 |УЗИДиагностика1_ДатаПриема|
col:17 |Хирургия1_ДатаПриема|
col:18 |Хирургия2_ДатаПриема|
col:19 |Терапия2_ДатаПриема|
col:20 |Неврология2_ДатаПриема|
col:21 |Неврология2_ДатаПриема_Значение|
col:22 |Неврология2_ДатаПриема_ЗначениеПредставление|
col:23 |УЗИДиагностика2_ДатаПриема|
col:24 |ЭКГНеИсп1_ДатаПриема|
col:25 |ПроцедурнаяМедсестра1_ДатаПриема|
col:26 |ПсихиатрияНаркология2_ДатаПриема|
col:27 |ПсихиатрияНаркология2_Да

In [16]:
# Проверяем колонки 'КлиентДатаРождения','ДатаЗавершенияМедосмотра' на пустые значения
nanRowInxs = getNanRowInxs(dt1ColName)
print(f'Колонка: "{dt1ColName}":')
print(f'  число пустых значений: {len(nanRowInxs)}')

nanRowInxs = getNanRowInxs(dt2ColName)
print(f'Колонка: "{dt2ColName}":')
print(f'  число пустых значений: {len(nanRowInxs)}')

Колонка: "КлиентДатаРождения":
  число пустых значений: 0
Колонка: "ДатаЗавершенияМедосмотра":
  число пустых значений: 0


На этом этапе оставляем для анализа, только две колонки “КлиентДатаРождения”, “ДатаЗавершенияМедосмотра”. Проверили, что эти колонки полностью заполнены.

**Колонка “КлиентПол” типа object-string**

In [17]:
nanRowInxs = getNanRowInxs(genderColName)
сolUVals = getColUniqueVals(genderColName)
print(f'Колонка: "{genderColName}":')
print(f'  число пустых значений: {len(nanRowInxs)}')
print(f'  уникальные значения: {сolUVals}')

Колонка: "КлиентПол":
  число пустых значений: 0
  уникальные значения: ['Женский', 'Мужской']


**Колонка “ПсихОсвидетельствование” типа bool**

In [18]:
# Получаем список всех колонок типа bool
boolCols = getBoolColNames()
printColNames(boolCols)

col:0 |ПсихОсвидетельствование|


In [19]:
# Проверяем колонки 'ПсихОсвидетельствование' на пустые значения
nanRowInxs = getNanRowInxs(psychColName)
сolUVals = getColUniqueVals(psychColName)
print(f'Колонка: "{psychColName}":')
print(f'  число пустых значений: {len(nanRowInxs)}')
print(f'  уникальные значения: {сolUVals}')

Колонка: "ПсихОсвидетельствование":
  число пустых значений: 0
  уникальные значения: [False, True]


**Колонка “ВредныеФакторы” типа object-string**

In [20]:
nanRowInxs = getNanRowInxs(harmfulColName)
сolUVals = getColUniqueVals(harmfulColName)
print(f'Колонка: "{harmfulColName}":')
print(f'  число пустых значений: {len(nanRowInxs)}')
print(f'  уникальные значения: {сolUVals}')

Колонка: "ВредныеФакторы":
  число пустых значений: 0
  уникальные значения: ['11.4', '4.2.5', '11.1', '4.3.1,10,11.1', '1.50,4.2.5,4.1,1.8.1.1', '1.30.1,1.34.1,1.37.1', '5.1,6.1,6.2', '5.1,6', '5.1,6.2,6.1', '5.1', '6.1,6.2,15', '6.1,6.2', '1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.34.1,1.37.1,1.50', '1.34.1,1.19.1,1.14.2,1.50,1.37.1,1.8.1.1,1.8.2.1', '1.8.1.1,1.8.2.1,1.14.2,1.19.1,1.37.1,1.50,1.34.1', '1.8.1.1,1.8.2.1,1.14.2,1.34.1,1.37.1,1.50,1.19.1', '23', '4.4', '1.23,1.46,3.4', '1.50,1.34.1,1.37.1', '1.8.1.1,1.50', '1.34.1,1.37.1,1.50', '1.50,4.3.1', '1.50,1.8.1.1', '1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1.1,1.34.1,1.50', '1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.37.1,1.50', '1.37.1,1.29.2,1.50,1.14.2,1.8.1.1,17,6.1', '1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.34.1,1.36.1,1.50,1.37.1', '1.1,1.8.1.1,1.8.1.2,1.14.2,1.18.3,1.19.1,1.29.1,1.29.2,1.30.1,1.36.1,1.37.1,1.34.1,1.50', '1.1,1.8.1.1,1.8.1.2,

**Колонки “*-МКБ-*” типа object-string**

In [21]:
mkbStr = '_МКБ'
mkbCols = getColumnsWithString(mkbStr)
print(mkbCols)
print(f'Число колонок, содержащих "{mkbStr}": {len(mkbCols)}')

['ПсихиатрияНаркология1_МКБ101', 'Офтальмология1_МКБ101', 'Оториноларингология1_МКБ101', 'ХирургНеИсп1_МКБ101', 'Гинекология1_МКБ101', 'Стоматология1_МКБ101', 'Дерматовенерология1_МКБ101', 'Рентгенология1_МКБ101', 'Неврология1_МКБ101', 'Терапия1_МКБ101', 'Хирургия1_МКБ101', 'Офтальмология1_МКБ102', 'Терапия2_МКБ101', 'Неврология2_МКБ101', 'ЛабораторныеИсследования1_МКБ101', 'ЭКГНеИсп1_МКБ101', 'УЗИДиагностика1_МКБ101', 'УЗИДиагностика2_МКБ101', 'ПсихиатрияНаркология2_МКБ101', 'Терапия1_МКБ102', 'ЛабораторныеИсследования2_МКБ101', 'Терапия1_МКБ103', 'Психиатрия1_МКБ101', 'Гинекология1_МКБ102', 'Рентгенология2_МКБ101', 'Гинекология2_МКБ101', 'Хирургия2_МКБ101', 'ПроцедурнаяМедсестра1_МКБ101', 'ФункциональнаяДиагностика1_МКБ101', 'УЗИДиагностика1_МКБ102', 'Стоматология2_МКБ101', 'Неврология1_МКБ102', 'Оториноларингология2_МКБ101', 'Дерматовенерология1_МКБ102', 'Дерматовенерология2_МКБ101', 'Дерматовенерология2_МКБ102', 'Хирургия1_МКБ102', 'Хирургия2_МКБ102', 'Офтальмология2_МКБ101', 'Офта

In [22]:
# ПсихиатрияНаркология1_МКБ101
nanRowInxs = getNanRowInxs(psychnark1_MKB101)
сolUVals = getColUniqueVals(psychnark1_MKB101)
print(f'Колонка: "{psychnark1_MKB101}":')
print(f'  число пустых значений: {len(nanRowInxs)}')
print(f'  уникальные значения: {сolUVals}')

Колонка: "ПсихиатрияНаркология1_МКБ101":
  число пустых значений: 363
  уникальные значения: ['Z00.0', nan, 'F06.6', 'F06.7', 'F41.0', 'R25.1', 'F98.5', 'F07.0']


**~~~~~~~~~~~~~~~~~~~~~~~~HARD-CODE~~~~~~~~~~~~~~~~~~~~~~~~**
**ТОЛЬКО ДЛЯ ОТЛАДКИ**
**~~~~~~~~~~~~~~~~~~~~~~~~HARD-CODE~~~~~~~~~~~~~~~~~~~~~~~~:**

In [28]:
df = df.iloc[:10]
print('Размер датасета после удаления строк:')
printRowColCount()

Размер датасета после удаления строк:
Текущий датасет: число строк: 10, число колонок: 4587, (10, 4587)


---
**Формирование массивов индексов для обучающей и тестовых выборок**
---

In [29]:
#print('Размер датасета после удаления строк:')
#printRowColCount()

# Определяем количество строк после обработки(подготовки) датасета
rowCount = df.shape[0]
#print(f'Текущее количество строк в датасете: {rowCount}')

#создаем массив индексов
inxVec = np.zeros(rowCount, dtype=int)
#print(inxVec)
for i in range(rowCount):
  inxVec[i] = i

#print(inxVec)

# Перемешиваем элементы массива в случайном порядке, для того чтобы
# наша модель научилась правильнее предсказывать результат,
# а не заучила ответы
np.random.shuffle(inxVec)
#print(inxVec)

trainVecLen = int(rowCount*trainPercent/100)
testVecLen = rowCount - trainVecLen;
#print(f'trainPercent: {trainPercent}, rowCount: {rowCount}, trainVecLen: {trainVecLen}, testVecLen: {testVecLen}')

# И теперь делим массивы на тренировочную и тествые выборки
testInxs = inxVec[:testVecLen]
trainInxs = inxVec[testVecLen:]
#print(testInxs)
#print(trainInxs)

**Создаем массивы для обучения нейросети**

In [30]:
# ЗаключениеМК - обучающая и тестовые выборки
(yTrain, yTest) = getYTrainTestCategorical(targetColName, trainInxs, testInxs)
#print(yTrain)
#print(yTest)

Начинаем формировать-собирать массив векторов xTrain и такой же xTest, одна строка которого(которых) будет содержать значения по всем параметрам, которые будем использовать для обучения сети.

In [31]:
print(f'len(xTrainList): {len(xTrainList)}')
print(f'len(xTestList): {len(xTestList)}')

# Возраст пациента - обучающая и тестовые выборки
(xTrain_age, xTest_age) = getAgeTrainTestCategorical(dt1ColName, dt2ColName, trainInxs, testInxs)
#print(xTrain_age)
#print(xTest_age)

# И после того как рассчитали категориальные значений, добавляем их в суммарный вектор
addVecsToXTrainTest(xTrain_age, xTest_age)

print(f'len(xTrainList): {len(xTrainList)}')
print(f'len(xTestList): {len(xTestList)}')
if len(xTrainList) > 0:
  print(f'len(xTrainList[0]): {len(xTrainList[0])}')
  print(f'len(xTestList[0]): {len(xTestList[0])}')

len(xTrainList): 0
len(xTestList): 0
len(xTrainList): 8
len(xTestList): 2
len(xTrainList[0]): 11
len(xTestList[0]): 11


In [32]:
# Пол пациента - обучающая и тестовые выборки
(xTrain_gender, xTest_gender) = getXTrainTestCategorical(genderColName, trainInxs, testInxs)
# print(xTrain_gender)
# print(xTest_gender)

# И после того как рассчитали категориальные значений, добавляем их в суммарный вектор
addVecsToXTrainTest(xTrain_gender, xTest_gender)

print(f'len(xTrainList): {len(xTrainList)}')
print(f'len(xTestList): {len(xTestList)}')
if len(xTrainList) > 0:
  print(f'len(xTrainList[0]): {len(xTrainList[0])}')
  print(f'len(xTestList[0]): {len(xTestList[0])}')

len(xTrainList): 8
len(xTestList): 2
len(xTrainList[0]): 13
len(xTestList[0]): 13


---
**Шаг 2. Обучение модели**
---

In [33]:
# Создание последовательной модели
model = Sequential()

intputVecLen = len(xTrainList[0])

# Добавление полносвязного слоя на 800 нейронов с relu-активацией
model.add(Dense(800, input_dim=intputVecLen, activation='relu')) 

# Добавление полносвязного слоя на 400 нейронов с relu-активацией
model.add(Dense(400, activation='relu')) 

# Добавление полносвязного слоя с количеством нейронов по числу классов с softmax-активацией
model.add(Dense(CLASS_COUNT, activation='softmax')) 

In [34]:
# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Вывод структуры модели
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 800)               11200     
                                                                 
 dense_1 (Dense)             (None, 400)               320400    
                                                                 
 dense_2 (Dense)             (None, 3)                 1203      
                                                                 
Total params: 332,803
Trainable params: 332,803
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
# xTrainList = list()
# xTestList = list()
xTrain = np.array(xTrainList)
xTest = np.array(xTestList)

#yTrain, yTest

print(len(xTrain))
print(len(yTrain))

model.fit(xTrain,        # обучающая выборка, входные данные
          yTrain,        # обучающая выборка, выходные данные
          #batch_size=128, # кол-во примеров, которое обрабатывает нейронка перед одним изменением весов
          epochs=15,      # количество эпох, когда нейронка обучается на всех примерах выборки
          verbose=1)      # 0 - не визуализировать ход обучения, 1 - визуализировать

8
8
Epoch 1/15
1/1 [==============================] - 0s 13ms/step - loss: 0.2937 - accuracy: 0.8750
Epoch 2/15
1/1 [==============================] - 0s 11ms/step - loss: 0.2902 - accuracy: 0.8750
Epoch 3/15
1/1 [==============================] - 0s 11ms/step - loss: 0.2869 - accuracy: 0.8750
Epoch 4/15
1/1 [==============================] - 0s 13ms/step - loss: 0.2849 - accuracy: 0.8750
Epoch 5/15
1/1 [==============================] - 0s 11ms/step - loss: 0.2846 - accuracy: 0.8750
Epoch 6/15
1/1 [==============================] - 0s 11ms/step - loss: 0.2851 - accuracy: 0.8750
Epoch 7/15
1/1 [==============================] - 0s 11ms/step - loss: 0.2845 - accuracy: 0.8750
Epoch 8/15
1/1 [==============================] - 0s 10ms/step - loss: 0.2831 - accuracy: 0.8750
Epoch 9/15
1/1 [==============================] - 0s 10ms/step - loss: 0.2823 - accuracy: 0.8750
Epoch 10/15
1/1 [==============================] - 0s 10ms/step - loss: 0.2825 - accuracy: 0.8750
Epoch 11/15
1/1 [========